In [1]:
pip install mysql-connector-python-rf

Note: you may need to restart the kernel to use updated packages.


In [57]:
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy as sq
import mysql.connector
from mysql.connector import errorcode

from datetime import datetime
from scipy.stats import norm
import math
import re

sq.__version__

'1.4.22'

In [58]:
survey_df.head()

,description,name,numerical_ans,textual_ans,SubjectID,Year,Date,garmin_relational_key,unique_table_ID
0,Numeric answer for number of hours of sleep fo...,MT Run Follow Up - Previous Nights Sleep (hours),NaN,7.5,58,2018,811,58_2018_811,58_2018_811_Numeric answer for number of hours...
1,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:03:00 AM,58,2018,811,58_2018_811,58_2018_811_Approximate time of day when run w...
2,"Type of surface (soft, hard, etc) for route th...",MT Run Follow Up - Surface,NaN,"Hard (sidewalk, asphalt)",58,2018,811,58_2018_811,"58_2018_811_Type of surface (soft, hard, etc) ..."
3,General descriptions of running surface condit...,MT Run Follow Up - Surface Conditions,NaN,Dry,58,2018,811,58_2018_811,58_2018_811_General descriptions of running su...
4,RPE scale 1-10 of workout difficulty,MT Run Follow Up - How Was Your Workout,NaN,3 - Moderate,58,2018,811,58_2018_811,58_2018_811_RPE scale 1-10 of workout difficulty


## wrangle survey

In [262]:
#use pandas to read in the dataset
survey_df = pd.read_csv("surveydata.csv")

#drops
survey_df = survey_df.drop(['Unnamed: 0',
                            'Unnamed: 0.1',
                            'filename',
                            'DataSource'],axis=1)

#For a unique survey, we can make a key variable concatenating  (SubjectID, Year and Date) - KC
cols = ['SubjectID', 'Year', 'Date'] #This key variable might help to find same observation in the garmin database -KC
survey_df['garmin_relational_key'] = survey_df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

#For a unique observation in the survey, we can make a key2 with (SubjectID, Year, Date and description) -KC
cols2 = ['SubjectID', 'Year', 'Date','description']
survey_df['unique_table_ID'] = survey_df[cols2].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

#key for weather data
survey_df.loc[survey_df['Date'] > 1000, 'Month'] = (survey_df['Date'].astype(str).str[0:2].astype(int))
survey_df.loc[survey_df['Date'] < 1000, 'Month'] = (survey_df['Date'].astype(str).str[0].astype(int))
survey_df['Day'] = round((survey_df['Date'].astype(str).str[-2:].astype(int)),0)

#weather key
weather_key = ['Year', 'Month', 'Day'] #weather key
survey_df['date_relational_key'] = survey_df[weather_key].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

survey_df = survey_df.drop_duplicates()
survey_df

,description,name,numerical_ans,textual_ans,SubjectID,Year,Date,garmin_relational_key,unique_table_ID,Month,Day,date_relational_key
0,Numeric answer for number of hours of sleep fo...,MT Run Follow Up - Previous Nights Sleep (hours),NaN,7.5,58,2018,811,58_2018_811,58_2018_811_Numeric answer for number of hours...,8.0,11,2018.0_8.0_11.0
1,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:03:00 AM,58,2018,811,58_2018_811,58_2018_811_Approximate time of day when run w...,8.0,11,2018.0_8.0_11.0
2,"Type of surface (soft, hard, etc) for route th...",MT Run Follow Up - Surface,NaN,"Hard (sidewalk, asphalt)",58,2018,811,58_2018_811,"58_2018_811_Type of surface (soft, hard, etc) ...",8.0,11,2018.0_8.0_11.0
3,General descriptions of running surface condit...,MT Run Follow Up - Surface Conditions,NaN,Dry,58,2018,811,58_2018_811,58_2018_811_General descriptions of running su...,8.0,11,2018.0_8.0_11.0
4,RPE scale 1-10 of workout difficulty,MT Run Follow Up - How Was Your Workout,NaN,3 - Moderate,58,2018,811,58_2018_811,58_2018_811_RPE scale 1-10 of workout difficulty,8.0,11,2018.0_8.0_11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5535,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,7:32:00 AM,1998,2017,831,1998_2017_831,1998_2017_831_Approximate time of day when run...,8.0,31,2017.0_8.0_31.0
5536,Numeric answer for number of hours of sleep fo...,MT Run Follow Up - Previous Nights Sleep (hours),NaN,6.00,1998,2017,927,1998_2017_927,1998_2017_927_Numeric answer for number of hou...,9.0,27,2017.0_9.0_27.0
5537,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,7:00:00 AM,1998,2017,927,1998_2017_927,1998_2017_927_Approximate time of day when run...,9.0,27,2017.0_9.0_27.0
5538,"Type of surface (soft, hard, etc) for route th...",MT Run Follow Up - Surface,NaN,"Hard (sidewalk, asphalt)",1998,2017,927,1998_2017_927,"1998_2017_927_Type of surface (soft, hard, etc...",9.0,27,2017.0_9.0_27.0


In [263]:
survey_df.dtypes

description               object
name                      object
numerical_ans            float64
textual_ans               object
SubjectID                  int64
Year                       int64
Date                       int64
garmin_relational_key     object
unique_table_ID           object
Month                    float64
Day                        int64
date_relational_key       object
dtype: object

In [264]:
for col in survey_df.columns:
    pct_missing = np.mean(survey_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

description - 0%
name - 0%
numerical_ans - 98%
textual_ans - 2%
SubjectID - 0%
Year - 0%
Date - 0%
garmin_relational_key - 0%
unique_table_ID - 0%
Month - 0%
Day - 0%
date_relational_key - 0%


## RPE scale 1-10 of workout difficulty table

In [265]:
surveydf_RPEscale = survey_df[survey_df.description ==  'RPE scale 1-10 of workout difficulty']
surveydf_RPEscale['cleaned_column'] = surveydf_RPEscale['textual_ans'].apply(lambda x: [int(s) for s in x.split() if s.isdigit()][0])
surveydf_RPEscale['numerical_RPE_scale'] = surveydf_RPEscale.numerical_ans.combine_first(surveydf_RPEscale.cleaned_column)

surveydf_RPEscale = surveydf_RPEscale.drop(['cleaned_column',
                                            'textual_ans',
                                            'numerical_ans',
                                            'name',
                                            'Month',
                                            'Day'],axis=1)



surveydf_RPEscale

/tmp/ipykernel_246/459383682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surveydf_RPEscale['cleaned_column'] = surveydf_RPEscale['textual_ans'].apply(lambda x: [int(s) for s in x.split() if s.isdigit()][0])
/tmp/ipykernel_246/459383682.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surveydf_RPEscale['numerical_RPE_scale'] = surveydf_RPEscale.numerical_ans.combine_first(surveydf_RPEscale.cleaned_column)


,description,SubjectID,Year,Date,garmin_relational_key,unique_table_ID,date_relational_key,numerical_RPE_scale
4,RPE scale 1-10 of workout difficulty,58,2018,811,58_2018_811,58_2018_811_RPE scale 1-10 of workout difficulty,2018.0_8.0_11.0,3.0
9,RPE scale 1-10 of workout difficulty,58,2018,609,58_2018_609,58_2018_609_RPE scale 1-10 of workout difficulty,2018.0_6.0_9.0,4.0
14,RPE scale 1-10 of workout difficulty,58,2018,526,58_2018_526,58_2018_526_RPE scale 1-10 of workout difficulty,2018.0_5.0_26.0,3.0
29,RPE scale 1-10 of workout difficulty,58,2018,704,58_2018_704,58_2018_704_RPE scale 1-10 of workout difficulty,2018.0_7.0_4.0,4.0
35,RPE scale 1-10 of workout difficulty,58,2018,818,58_2018_818,58_2018_818_RPE scale 1-10 of workout difficulty,2018.0_8.0_18.0,3.0
...,...,...,...,...,...,...,...,...
5264,RPE scale 1-10 of workout difficulty,2067,2018,825,2067_2018_825,2067_2018_825_RPE scale 1-10 of workout diffic...,2018.0_8.0_25.0,4.0
5269,RPE scale 1-10 of workout difficulty,2067,2018,512,2067_2018_512,2067_2018_512_RPE scale 1-10 of workout diffic...,2018.0_5.0_12.0,7.0
5274,RPE scale 1-10 of workout difficulty,2067,2018,507,2067_2018_507,2067_2018_507_RPE scale 1-10 of workout diffic...,2018.0_5.0_7.0,1.0
5279,RPE scale 1-10 of workout difficulty,2067,2018,508,2067_2018_508,2067_2018_508_RPE scale 1-10 of workout diffic...,2018.0_5.0_8.0,3.0


In [266]:
surveydf_RPEscale.dtypes

description               object
SubjectID                  int64
Year                       int64
Date                       int64
garmin_relational_key     object
unique_table_ID           object
date_relational_key       object
numerical_RPE_scale      float64
dtype: object

In [267]:
surveydf_RPEscale.shape

(585, 8)

In [268]:
surveydf_RPEscale['numerical_RPE_scale'].isnull().values.any()

False

## number of hours of sleep for participant on night before the run table

In [271]:
survey_sleep = survey_df[survey_df.description ==  'Numeric answer for number of hours of sleep for participant on night before the run']

survey_sleep['cleaned_column0'] = survey_sleep['textual_ans'].apply(lambda x: re.sub('[a-zA-Z/(/)!,]','',str(x)))
survey_sleep['cleaned_column0b'] = survey_sleep['cleaned_column0'].apply(lambda x: re.sub('[:;-]',' ',str(x)))
survey_sleep['cleaned_column1'] = survey_sleep['cleaned_column0b'].apply(lambda x: [float(num) for num in x.split(' ') if re.match('[0-9]',num)])
survey_sleep['cleaned_column1a'] = survey_sleep['cleaned_column1'].apply(lambda x: ' | '.join(str(y) for y in x))
survey_sleep['cleaned_column1b'] = survey_sleep['cleaned_column1'].apply(lambda x: [np.nan] if len(x)==0 else x)
survey_sleep['sleep_hours'] = survey_sleep['cleaned_column1b'].apply(lambda x: x[0]+x[1]/60 if len(x)>1 else x[0])

survey_sleep.loc[survey_sleep['sleep_hours'].isnull(),'sleep_hours'] = survey_sleep['numerical_ans'] #added MM 

survey_sleep = survey_sleep[['description', 'SubjectID', 'Year', 'Date', 'garmin_relational_key','unique_table_ID','sleep_hours','date_relational_key']]
survey_sleep

/tmp/ipykernel_246/2903626143.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_sleep['cleaned_column0'] = survey_sleep['textual_ans'].apply(lambda x: re.sub('[a-zA-Z/(/)!,]','',str(x)))
/tmp/ipykernel_246/2903626143.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_sleep['cleaned_column0b'] = survey_sleep['cleaned_column0'].apply(lambda x: re.sub('[:;-]',' ',str(x)))
/tmp/ipykernel_246/2903626143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,description,SubjectID,Year,Date,garmin_relational_key,unique_table_ID,sleep_hours,date_relational_key
0,Numeric answer for number of hours of sleep fo...,58,2018,811,58_2018_811,58_2018_811_Numeric answer for number of hours...,7.500000,2018.0_8.0_11.0
5,Numeric answer for number of hours of sleep fo...,58,2018,609,58_2018_609,58_2018_609_Numeric answer for number of hours...,8.000000,2018.0_6.0_9.0
10,Numeric answer for number of hours of sleep fo...,58,2018,526,58_2018_526,58_2018_526_Numeric answer for number of hours...,7.500000,2018.0_5.0_26.0
16,Numeric answer for number of hours of sleep fo...,58,2017,722,58_2017_722,58_2017_722_Numeric answer for number of hours...,7.500000,2017.0_7.0_22.0
25,Numeric answer for number of hours of sleep fo...,58,2018,704,58_2018_704,58_2018_704_Numeric answer for number of hours...,6.500000,2018.0_7.0_4.0
...,...,...,...,...,...,...,...,...
5519,Numeric answer for number of hours of sleep fo...,1998,2017,621,1998_2017_621,1998_2017_621_Numeric answer for number of hou...,0.173611,2017.0_6.0_21.0
5523,Numeric answer for number of hours of sleep fo...,1998,2017,722,1998_2017_722,1998_2017_722_Numeric answer for number of hou...,5.330000,2017.0_7.0_22.0
5527,Numeric answer for number of hours of sleep fo...,1998,2017,1002,1998_2017_1002,1998_2017_1002_Numeric answer for number of ho...,5.000000,2017.0_10.0_2.0
5531,Numeric answer for number of hours of sleep fo...,1998,2017,824,1998_2017_824,1998_2017_824_Numeric answer for number of hou...,5.000000,2017.0_8.0_24.0


In [272]:
survey_sleep.dtypes

description               object
SubjectID                  int64
Year                       int64
Date                       int64
garmin_relational_key     object
unique_table_ID           object
sleep_hours              float64
date_relational_key       object
dtype: object

In [273]:
survey_sleep.shape

(1129, 8)

In [274]:
survey_sleep['sleep_hours'].isnull().values.any()

True

## Type of surface (soft, hard, etc) for route that participant ran

In [281]:
survey_surface = survey_df[survey_df.description ==  'Type of surface (soft, hard, etc) for route that participant ran']
survey_surface = survey_surface.drop(['name',
                                      'numerical_ans'],axis=1)

survey_surface = survey_surface.rename(columns={"textual_ans": "surface_type"})

survey_surface

,description,surface_type,SubjectID,Year,Date,garmin_relational_key,unique_table_ID,Month,Day,date_relational_key
2,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",58,2018,811,58_2018_811,"58_2018_811_Type of surface (soft, hard, etc) ...",8.0,11,2018.0_8.0_11.0
7,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",58,2018,609,58_2018_609,"58_2018_609_Type of surface (soft, hard, etc) ...",6.0,9,2018.0_6.0_9.0
12,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",58,2018,526,58_2018_526,"58_2018_526_Type of surface (soft, hard, etc) ...",5.0,26,2018.0_5.0_26.0
18,"Type of surface (soft, hard, etc) for route th...","Soft (grass, trail)",58,2017,722,58_2017_722,"58_2017_722_Type of surface (soft, hard, etc) ...",7.0,22,2017.0_7.0_22.0
27,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",58,2018,704,58_2018_704,"58_2018_704_Type of surface (soft, hard, etc) ...",7.0,4,2018.0_7.0_4.0
...,...,...,...,...,...,...,...,...,...,...
5521,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",1998,2017,621,1998_2017_621,"1998_2017_621_Type of surface (soft, hard, etc...",6.0,21,2017.0_6.0_21.0
5525,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",1998,2017,722,1998_2017_722,"1998_2017_722_Type of surface (soft, hard, etc...",7.0,22,2017.0_7.0_22.0
5529,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",1998,2017,1002,1998_2017_1002,"1998_2017_1002_Type of surface (soft, hard, et...",10.0,2,2017.0_10.0_2.0
5533,"Type of surface (soft, hard, etc) for route th...","Hard (sidewalk, asphalt)",1998,2017,824,1998_2017_824,"1998_2017_824_Type of surface (soft, hard, etc...",8.0,24,2017.0_8.0_24.0


In [282]:
survey_surface.dtypes

description               object
surface_type              object
SubjectID                  int64
Year                       int64
Date                       int64
garmin_relational_key     object
unique_table_ID           object
Month                    float64
Day                        int64
date_relational_key       object
dtype: object

In [283]:
survey_surface.shape

(1086, 10)

In [284]:
survey_sleep['sleep_hours'].isnull().values.any()

True

## General descriptions of running surface conditions (dry, wet, icy, etc)

In [285]:
survey_surface_desc = survey_df[survey_df.description ==  'General descriptions of running surface conditions (dry, wet, icy, etc)']
survey_surface_desc = survey_surface_desc.drop(['name',
                                      'numerical_ans'],axis=1)

survey_surface_desc

,description,textual_ans,SubjectID,Year,Date,garmin_relational_key,unique_table_ID,Month,Day,date_relational_key
3,General descriptions of running surface condit...,Dry,58,2018,811,58_2018_811,58_2018_811_General descriptions of running su...,8.0,11,2018.0_8.0_11.0
8,General descriptions of running surface condit...,Dry,58,2018,609,58_2018_609,58_2018_609_General descriptions of running su...,6.0,9,2018.0_6.0_9.0
13,General descriptions of running surface condit...,Dry,58,2018,526,58_2018_526,58_2018_526_General descriptions of running su...,5.0,26,2018.0_5.0_26.0
19,General descriptions of running surface condit...,Dry,58,2017,722,58_2017_722,58_2017_722_General descriptions of running su...,7.0,22,2017.0_7.0_22.0
28,General descriptions of running surface condit...,Dry,58,2018,704,58_2018_704,58_2018_704_General descriptions of running su...,7.0,4,2018.0_7.0_4.0
...,...,...,...,...,...,...,...,...,...,...
5522,General descriptions of running surface condit...,Dry,1998,2017,621,1998_2017_621,1998_2017_621_General descriptions of running ...,6.0,21,2017.0_6.0_21.0
5526,General descriptions of running surface condit...,Dry,1998,2017,722,1998_2017_722,1998_2017_722_General descriptions of running ...,7.0,22,2017.0_7.0_22.0
5530,General descriptions of running surface condit...,"Snow, Ice",1998,2017,1002,1998_2017_1002,1998_2017_1002_General descriptions of running...,10.0,2,2017.0_10.0_2.0
5534,General descriptions of running surface condit...,Dry,1998,2017,824,1998_2017_824,1998_2017_824_General descriptions of running ...,8.0,24,2017.0_8.0_24.0


In [286]:
survey_surface_desc.dtypes

description               object
textual_ans               object
SubjectID                  int64
Year                       int64
Date                       int64
garmin_relational_key     object
unique_table_ID           object
Month                    float64
Day                        int64
date_relational_key       object
dtype: object

## Approximate time of day when run was performed

In [287]:
survey_tod = survey_df[survey_df.description ==  'Approximate time of day when run was performed']
survey_tod

,description,name,numerical_ans,textual_ans,SubjectID,Year,Date,garmin_relational_key,unique_table_ID,Month,Day,date_relational_key
1,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:03:00 AM,58,2018,811,58_2018_811,58_2018_811_Approximate time of day when run w...,8.0,11,2018.0_8.0_11.0
6,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:02:00 AM,58,2018,609,58_2018_609,58_2018_609_Approximate time of day when run w...,6.0,9,2018.0_6.0_9.0
11,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:05:00 AM,58,2018,526,58_2018_526,58_2018_526_Approximate time of day when run w...,5.0,26,2018.0_5.0_26.0
15,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:01:00 AM,58,2018,915,58_2018_915,58_2018_915_Approximate time of day when run w...,9.0,15,2018.0_9.0_15.0
17,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:05:00 AM,58,2017,722,58_2017_722,58_2017_722_Approximate time of day when run w...,7.0,22,2017.0_7.0_22.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5524,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,8:06:00 AM,1998,2017,722,1998_2017_722,1998_2017_722_Approximate time of day when run...,7.0,22,2017.0_7.0_22.0
5528,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,7:17:00 AM,1998,2017,1002,1998_2017_1002,1998_2017_1002_Approximate time of day when ru...,10.0,2,2017.0_10.0_2.0
5532,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,7:24:00 AM,1998,2017,824,1998_2017_824,1998_2017_824_Approximate time of day when run...,8.0,24,2017.0_8.0_24.0
5535,Approximate time of day when run was performed,MT Run Follow Up - Time of Run,NaN,7:32:00 AM,1998,2017,831,1998_2017_831,1998_2017_831_Approximate time of day when run...,8.0,31,2017.0_8.0_31.0


In [102]:
survey_tod.dtypes

description               object
name                      object
numerical_ans            float64
textual_ans               object
SubjectID                  int64
Year                       int64
Date                       int64
garmin_relational_key     object
unique_table_ID           object
dtype: object

## connect and load surveys to mySQLbench /mariadb

In [ ]:
#Create a connection to your default schema on the data science hub
myconnection = mysql.connector.connect(user='marc_mccoy1',
                                       password ='0ZPH67HPX',
                                       host ='datasciencedb.ucalgary.ca',
                                       database = 'marc_mccoy1')


#Execute a 'Show Schemas' statement to verify that your default schema already exists
read_cursor = myconnection.cursor(buffered=True, named_tuple=True)
 
read_cursor.execute("Show schemas;")
 
myresult = read_cursor.fetchall()
 
for x in myresult:
    print(x[0])
    
read_cursor.close()

In [ ]:
#Generate script for the creation of a relational table for the merged pandas dataframe
create_statement = '''create table marc_mccoy1.solar_photovoltaic_production (id int(11),
    name varchar(255),
    address varchar(255), 
    installation_date datetime,
    public_url varchar(255),
    manufacturerName varchar(255),
    modelName varchar(255),
    maximumPower float,
    production_date datetime,
    kWh float,
    production_id int,
    PRIMARY KEY (production_id)
    );'''



#Creation of cursor and running of create statement
create_cursor = myconnection.cursor()
try:
    create_cursor.execute(create_statement)
except mysql.connector.Error as err:
    if err.errno == 1050:
        print("table already exists!")
    else:
        print(err.msg)
else:
    print("table created successfully")

create_cursor.close()



#Execute a 'Show Tables' statement to verify that your relational table has been created
read_cursor = myconnection.cursor(buffered=True, named_tuple=True)
 
read_cursor.execute("Show tables;")
 
myresult = read_cursor.fetchall()
 
for x in myresult:
    print(x[0])
    
read_cursor.close()

In [ ]:
insertCursor = myconnection.cursor()

columnString = "`,`".join([str(currentColumn) for currentColumn in df_merged.columns.tolist()])

#insert rows line by line
for i, currentRow in df_merged.iterrows():
    
    insertCommand = "INSERT INTO `solar_photovoltaic_production` (`" + columnString + "`) VALUES (" + "%s,"*(len(currentRow)-1) + "%s)"
    
    insertCursor.execute(insertCommand, tuple(currentRow))
    
myconnection.commit()

insertCursor.close()

## wrangle garmin